https://www.psycopg.org/docs/

https://pythonru.com/biblioteki/vvedenie-v-postgresql-s-python-psycopg2

https://khashtamov.com/ru/postgresql-python-psycopg2/

https://dev-gang.ru/article/rabota-s-postgresql-v-python-xn8721sq0g/

https://docs.python.org/3/library/zipfile.html

https://docs.python.org/3/library/os.path.html

https://docs-python.ru/tutorial/operatsii-tekstovymi-strokami-str-python/

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import psycopg2

from datetime import date
from contextlib import closing

from Ozon_performance_class import Ozon_performance

In [2]:
class db_working:
    def __init__(self, db_access = """host=rc1b-itt1uqz8cxhs0c3d.mdb.yandexcloud.net\
                                    port=6432\
                                    sslmode=verify-full\
                                    dbname=market_db\
                                    user=sfedyusnin\
                                    password=Qazwsx123Qaz\
                                    target_session_attrs=read-write"""):        
        self.db_access = db_access
        
        # необходимые запросы к БД
        self.api_keys_resp = 'SELECT * FROM account_list'
        self.keys_dt_cols_resp = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'account_list'"
        self.an_dt_resp = 'SELECT * FROM analitics_data2'
        self.an_dt_cols_resp = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'analitics_data2'"
        
#         self.db_data = self.get_analitics_data()
        
    def test_db_connection(self):
        """
        Проверка доступа к БД
        """
        conn = psycopg2.connect(self.db_access)
        q = conn.cursor()
        q.execute('SELECT version()')
        print(q.fetchone())
        conn.close()
        
    def get_analitics_data(self):
        """
        Загружает таблицу из базы
        """
        print('Загрузка analitics_data')
        self.db_data = pd.read_sql(self.an_dt_resp, psycopg2.connect(self.db_access))
#         return self.db_data
    
    def get_last_date(self):
        """
        Возвращает последнюю дату записи в базе
        """
        db_data = self.db_data
        return db_data['data'].sort_values(ascending=False)[0]
    
    def get_keys(self):
        """
        Загружает из базы таблицу ключей
        """
        print('Загрузка api_keys')
        try:   
            return pd.read_sql(self.api_keys_resp, psycopg2.connect(self.db_access))
        except:
            print('Доступ к таблице запрещен')
             
    def extract_zips(self, path = r'./data/'):
        """
        Распаковывает все zip лежащие в пути path
        """
        zip_files = glob.glob(os.path.join(path, "*.zip"))
        for file in zip_files:
            print(f'Распаковка {file}')
            with zipfile.ZipFile(file) as zf:
                zf.extractall(path)
                
    def stat_read_trans(self, path, api_id=None, account_id=None):
        """
        Обрабатывает датасет
        """
        data = pd.read_csv(path, sep=';')
        data = data.reset_index()

        camp = data.keys()[-1].split(',')[0].split()[-1]
        data.columns = data[0:1].values.tolist()[0]
        data.drop(index=0, inplace=True)
        data.drop(data.tail(1).index, inplace=True)
        
        data['api_id'] = api_id
        data['account_id'] = account_id
        data['Кампания'] = camp

        data = data[data.columns[-1:].tolist()+data.columns[:-1].tolist()]
        data = data[data.columns.dropna()]
        data = data.dropna(axis=0, how='any', thresh=10)
        return(data)
                      
    def make_dataset(self, path=r'./data/statistics', api_id=None, account_id=None):
        """
        Собирает датасет
        """
        csv_files = glob.glob(os.path.join(path, "*.csv"))
        stat_data = []
        for file in csv_files:
            stat_data.append(self.stat_read_trans(file, api_id=api_id, account_id=account_id))
        dataset = pd.concat(stat_data, axis=0).reset_index().drop('index', axis=1)
        dataset['data'] = dataset[['Дата', 'День']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Дата', 'День'], inplace=True)
        dataset['name'] = dataset[['Наименование', 'Название товара']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Наименование', 'Название товара'], inplace=True)
        dataset['orders'] = dataset[['Количество', 'Заказы']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Количество', 'Заказы'], inplace=True)
        dataset['price'] = dataset[['Цена продажи', 'Цена товара (руб.)']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Цена продажи', 'Цена товара (руб.)'], inplace=True)
        dataset['revenue'] = dataset[['Выручка (руб.)', 'Стоимость, руб.']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Выручка (руб.)', 'Стоимость, руб.'], inplace=True)
        dataset['expense'] = dataset[['Расход (руб., с НДС)', 'Расход, руб.']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
        dataset.drop(columns=['Расход (руб., с НДС)', 'Расход, руб.'], inplace=True)

        dataset.rename(columns={'ID заказа':'order_id', 'Номер заказа': 'order_number', 'Ozon ID': 'ozon_id',
                                'Ozon ID рекламируемого товара': 'ozon_id_ad_sku', 'Артикул': 'articul',
                                'Ставка, %': 'search_price_perc', 'Ставка, руб.': 'search_price_rur',
                                'Тип страницы': 'pagetype', 'Условие показа': 'viewtype', 'Показы': 'views',
                                'Клики': 'clicks', 'CTR (%)': 'ctr', 'Средняя ставка за 1000 показов (руб.)': 'cpm',
                                'Заказы модели': 'orders_model', 'Выручка с заказов модели (руб.)': 'revenue_model',
                                'Тип условия': 'request_type', 'Платформа': 'platfrom', 'Охват': 'audience',
                                'Баннер': 'banner', 'Средняя ставка (руб.)': 'avrg_bid', 'Кампания':'actionnum'}, inplace=True)

        return dataset      

In [3]:
working = db_working()

In [4]:
working.test_db_connection()

('PostgreSQL 14.1 (Ubuntu 14.1-201-yandex.52142.672784f35a) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [5]:
working.get_analitics_data()
api_keys = working.get_keys()

Загрузка analitics_data
Загрузка api_keys
Доступ к таблице запрещен


In [6]:
db_data = working.db_data
last_date = str(working.get_last_date())

In [7]:
# for index, keys in api_keys[['client_id_performance', 'client_secret_performance', 'client_id']].iterrows():
#     client_id = keys[0]
#     client_secret = keys[1]
#     ozon = Ozon_performance(client_id, client_secret)
#     ozon.collect_data(date_from = last_date, date_to = str(date.today()),
#                       methods = {'statistics': True, 'phrases': False, 'attribution': False, 
#                                  'media': False,'product': False, 'daily': False})
#     ozon.save_data(path = r'./data/', methods = {'statistics': True, 'phrases': False, 'attribution': False, 
#                                                  'media': False,'product': False, 'daily': False})

client_id = '2471117-1649840283864@advertising.performance.ozon.ru'
client_secret = 'aydWhTlynReq4FiDELQhpWC2-j2kfE7n80uOO_ByHt-0A7zAlskxUQFyCa_cp33W0lPvsNZIzwXoSaxh2Q'

ozon = Ozon_performance(client_id, client_secret)
print(ozon.client_id)

Подключение успешно, токен получен
Найдено 19 кампаний
2471117-1649840283864@advertising.performance.ozon.ru


In [8]:
working.extract_zips(path = r'./data/statistics')

In [9]:
dataset = working.make_dataset(path=r'./data/statistics', api_id = ozon.client_id.split('@')[0])

In [10]:
dataset.columns

Index(['actionnum', 'order_id', 'order_number', 'ozon_id', 'ozon_id_ad_sku',
       'articul', 'search_price_perc', 'search_price_rur', 'api_id',
       'account_id', 'sku', 'pagetype', 'viewtype', 'views', 'clicks', 'ctr',
       'cpm', 'orders_model', 'revenue_model', 'request_type', 'platfrom',
       'audience', 'banner', 'avrg_bid', 'data', 'name', 'orders', 'price',
       'revenue', 'expense'],
      dtype='object')

In [11]:
dataset

,actionnum,order_id,order_number,ozon_id,ozon_id_ad_sku,articul,search_price_perc,search_price_rur,api_id,account_id,...,platfrom,audience,banner,avrg_bid,data,name,orders,price,revenue,expense
0,370873,749926556,01737316-0053,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,01.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
1,370873,751933273,08191834-0039,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,02.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
2,370873,754720770,01737316-0054,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,04.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
3,370873,756212849,40713186-0154,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
4,370873,755483634,0103473783-0008,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","0,00"
1432,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","21,00"
1433,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,06.07.2022,Беспроводная Видеоняня поворотная IP WiFi каме...,0,"7690,00","0,00","0,90"
1434,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,06.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","1,80"


In [12]:
db_data

,id,banner,pagetype,viewtype,platfrom,request_type,sku,name,order_id,order_number,...,price,orders,revenue_model,orders_model,revenue,expense,cpm,ctr,data,api_id
0,874555,None,None,None,None,None,None,Уличная беспроводная WiFi камера видеонаблюден...,667431230.0,23597329-0005,...,7990.0,1.0,NaN,NaN,7990.0,719.1,NaN,NaN,2022-05-02,None
1,874556,None,None,None,None,None,None,Беспроводная поворотная IP WiFi камера видеона...,668886837.0,29493179-0089,...,7190.0,1.0,NaN,NaN,7190.0,647.1,NaN,NaN,2022-05-03,None
2,874557,None,None,None,None,None,None,Беспроводная IP WiFi камера видеонаблюдения Iv...,668275286.0,40922410-0099,...,5390.0,1.0,NaN,NaN,5390.0,485.1,NaN,NaN,2022-05-03,None
3,874558,None,None,None,None,None,None,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,1851.3,NaN,NaN,2000-01-01,None
4,874559,None,Категории,nan,None,None,297666435.0,Уличная камера видеонаблюдения Ivideon Bullet ...,None,None,...,8390.0,0.0,0.0,0.0,0.0,0.3,100.00,0.00,2022-05-01,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269440,1301352,None,None,наколенники защитные взрослые,Мобильное приложение,Поисковый запрос,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,0.3,300.00,0.00,2022-07-05,2490357
269441,1301353,None,None,налокотники наколенники,Мобильное приложение,Поисковый запрос,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,0.3,300.00,0.00,2022-07-05,2490357
269442,1301354,None,None,налокотники наколенники,Десктоп,Поисковый запрос,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,1.5,300.00,0.00,2022-07-05,2490357
269443,1301355,None,None,экипировка для роликов детская,Мобильное приложение,Поисковый запрос,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,0.3,300.00,0.00,2022-07-05,2490357


In [13]:
db_data.columns

Index(['id', 'banner', 'pagetype', 'viewtype', 'platfrom', 'request_type',
       'sku', 'name', 'order_id', 'order_number', 'ozon_id', 'ozon_id_ad_sku',
       'articul', 'empty', 'account_id', 'views', 'clicks', 'audience',
       'exp_bonus', 'actionnum', 'avrg_bid', 'search_price_rur',
       'search_price_perc', 'price', 'orders', 'revenue_model', 'orders_model',
       'revenue', 'expense', 'cpm', 'ctr', 'data', 'api_id'],
      dtype='object')

In [14]:
list(dataset.columns)

['actionnum',
 'order_id',
 'order_number',
 'ozon_id',
 'ozon_id_ad_sku',
 'articul',
 'search_price_perc',
 'search_price_rur',
 'api_id',
 'account_id',
 'sku',
 'pagetype',
 'viewtype',
 'views',
 'clicks',
 'ctr',
 'cpm',
 'orders_model',
 'revenue_model',
 'request_type',
 'platfrom',
 'audience',
 'banner',
 'avrg_bid',
 'data',
 'name',
 'orders',
 'price',
 'revenue',
 'expense']

In [15]:
into_db = pd.concat([db_data, dataset], axis=0).reset_index().drop('index', axis=1).drop_duplicates(subset=list(dataset.columns), keep=False)

In [16]:
into_db

,id,banner,pagetype,viewtype,platfrom,request_type,sku,name,order_id,order_number,...,price,orders,revenue_model,orders_model,revenue,expense,cpm,ctr,data,api_id
0,874555.0,None,None,None,None,None,None,Уличная беспроводная WiFi камера видеонаблюден...,667431230.0,23597329-0005,...,7990.0,1.0,NaN,NaN,7990.0,719.1,NaN,NaN,2022-05-02,None
1,874556.0,None,None,None,None,None,None,Беспроводная поворотная IP WiFi камера видеона...,668886837.0,29493179-0089,...,7190.0,1.0,NaN,NaN,7190.0,647.1,NaN,NaN,2022-05-03,None
2,874557.0,None,None,None,None,None,None,Беспроводная IP WiFi камера видеонаблюдения Iv...,668275286.0,40922410-0099,...,5390.0,1.0,NaN,NaN,5390.0,485.1,NaN,NaN,2022-05-03,None
3,874558.0,None,None,None,None,None,None,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,1851.3,NaN,NaN,2000-01-01,None
4,874559.0,None,Категории,nan,None,None,297666435.0,Уличная камера видеонаблюдения Ivideon Bullet ...,None,None,...,8390.0,0.0,0.0,0.0,0.0,0.3,100.0,0.0,2022-05-01,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270876,NaN,NaN,Неизвестный тип страницы,NaN,NaN,NaN,420462204,Беспроводная Видеоняня IP WiFi камера видеонаб...,NaN,NaN,...,"5990,00",0,"0,00",0,"0,00","0,00","0,00","0,00",05.07.2022,2471117-1649840283864
270877,NaN,NaN,Неизвестный тип страницы,Радио- и видеоняни,NaN,NaN,420462204,Беспроводная Видеоняня IP WiFi камера видеонаб...,NaN,NaN,...,"5990,00",0,"0,00",0,"0,00","21,00","300,00","0,00",05.07.2022,2471117-1649840283864
270878,NaN,NaN,Неизвестный тип страницы,Радио- и видеоняни,NaN,NaN,420462196,Беспроводная Видеоняня поворотная IP WiFi каме...,NaN,NaN,...,"7690,00",0,"0,00",0,"0,00","0,90","300,00","0,00",06.07.2022,2471117-1649840283864
270879,NaN,NaN,Карточка товара,Радио- и видеоняни,NaN,NaN,420462204,Беспроводная Видеоняня IP WiFi камера видеонаб...,NaN,NaN,...,"5990,00",0,"0,00",0,"0,00","1,80","300,00","0,00",06.07.2022,2471117-1649840283864


In [108]:
# dataset['DATA'] = dataset[['Дата', 'День']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Дата', 'День'], inplace=True)
# dataset['name'] = dataset[['Наименование', 'Название товара']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Наименование', 'Название товара'], inplace=True)
# dataset['orders'] = dataset[['Количество', 'Заказы']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Количество', 'Заказы'], inplace=True)
# dataset['price'] = dataset[['Цена продажи', 'Цена товара (руб.)']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Цена продажи', 'Цена товара (руб.)'], inplace=True)
# dataset['revenue'] = dataset[['Выручка (руб.)', 'Стоимость, руб.']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Выручка (руб.)', 'Стоимость, руб.'], inplace=True)
# dataset['expence'] = dataset[['Расход (руб., с НДС)', 'Расход, руб.']].fillna('').apply(lambda x: x[0] if x[1] == '' else x[1], axis=1)
# dataset.drop(columns=['Расход (руб., с НДС)', 'Расход, руб.'], inplace=True)

# dataset.rename(columns={'ID заказа':'order_id', 'Номер заказа': 'order_number', 'Ozon ID': 'ozon_id',
#                         'Ozon ID рекламируемого товара': 'ozon_id_ad_sku', 'Артикул': 'articul',
#                         'Ставка, %': 'search_price_perc', 'Ставка, руб.': 'search_price_rur',
#                         'Тип страницы': 'pagetype', 'Условие показа': 'viewtype', 'Показы': 'views',
#                         'Клики': 'clicks', 'CTR (%)': 'ctr', 'Средняя ставка за 1000 показов (руб.)': 'cpm',
#                         'Заказы модели': 'orders_model', 'Выручка с заказов модели (руб.)': 'revenue_model',
#                         'Тип условия': 'request_type', 'Платформа': 'platfrom', 'Охват': 'audience',
#                         'Баннер': 'banner', 'Средняя ставка (руб.)': 'avrg_bid', 'Кампания':'actionnum'}, inplace=True)


In [155]:
dataset.columns

Index(['actionnum', 'order_id', 'order_number', 'ozon_id', 'ozon_id_ad_sku',
       'articul', 'search_price_perc', 'search_price_rur', 'api_id',
       'account_id', 'sku', 'pagetype', 'viewtype', 'views', 'clicks', 'ctr',
       'cpm', 'orders_model', 'revenue_model', 'request_type', 'platfrom',
       'audience', 'banner', 'avrg_bid', 'DATA', 'name', 'orders', 'price',
       'revenue', 'expence'],
      dtype='object')

In [156]:
dataset

,actionnum,order_id,order_number,ozon_id,ozon_id_ad_sku,articul,search_price_perc,search_price_rur,api_id,account_id,...,platfrom,audience,banner,avrg_bid,DATA,name,orders,price,revenue,expence
0,370873,749926556,01737316-0053,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,01.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
1,370873,751933273,08191834-0039,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,02.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
2,370873,754720770,01737316-0054,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,04.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
3,370873,756212849,40713186-0154,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
4,370873,755483634,0103473783-0008,297685245,297685245,NaN,"9,95","397,005",2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная IP WiFi камера видеонаблюдения Iv...,1,"3990,00","3990,00","397,005"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","0,00"
1432,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,05.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","21,00"
1433,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,06.07.2022,Беспроводная Видеоняня поворотная IP WiFi каме...,0,"7690,00","0,00","0,90"
1434,835659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471117-1649840283864,None,...,NaN,NaN,NaN,NaN,06.07.2022,Беспроводная Видеоняня IP WiFi камера видеонаб...,0,"5990,00","0,00","1,80"


In [84]:
working.get_analitics_data()

Загрузка analitics_data


In [85]:
db_data = working.db_data

In [87]:
db_data.columns

Index(['id', 'banner', 'pagetype', 'viewtype', 'platfrom', 'request_type',
       'sku', 'name', 'order_id', 'order_number', 'ozon_id', 'ozon_id_ad_sku',
       'articul', 'empty', 'account_id', 'views', 'clicks', 'audience',
       'exp_bonus', 'actionnum', 'avrg_bid', 'search_price_rur',
       'search_price_perc', 'price', 'orders', 'revenue_model', 'orders_model',
       'revenue', 'expense', 'cpm', 'ctr', 'data', 'api_id'],
      dtype='object')

In [91]:
from_xl = ['День', 'Баннер', 'Тип страницы', 'Условие показа', 'Платформа', 'Показы', 'Клики', 'CTR (%)', 'Охват',
'Средняя ставка за 1000 показов (руб.)', 'Расход (руб., с НДС)', 'Тип условия', 'sku', 'Название товара',
'Цена товара (руб.)', 'Заказы', 'Выручка (руб.)', 'Заказы модели', 'Выручка с заказов модели (руб.)',
'Дата', 'ID заказа', 'Номер заказа', 'Ozon ID', 'Ozon ID рекламируемого товара', 'Артикул',
'Наименование', 'Количество', 'Цена продажи', 'Стоимость, руб.', 'Ставка, %', 'Ставка, руб.',
'Расход, руб.', 'Unnamed: 1', 'Средняя ставка (руб.)', 'Расход за минусом бонусов (руб., с НДС)']

from_db = ['DATA', 'Banner', 'PageType', 'ViewType', 'Platfrom', 'Views', 'Clicks', 'CTR', 'Audience', 'CPM',
'Expense', 'Request_Type', 'SKU', 'Name', 'Price', 'Orders', 'Revenue', 'Orders_Model', 'Revenue_Model',
'DATA', 'Order_ID', 'Order_Number', 'Ozon_ID', 'Ozon_ID_Ad_SKU', 'Articul', 'Name', 'Orders', 'Price',
'Revenue', 'Search_price_perc', 'Search_price_RUR', 'Expense', 'EMPTY', 'Avrg_bid', 'exp_bonus']

mapping_data = dict(zip(from_xl, from_db))

In [92]:
mapping_data

{'День': 'DATA',
 'Баннер': 'Banner',
 'Тип страницы': 'PageType',
 'Условие показа': 'ViewType',
 'Платформа': 'Platfrom',
 'Показы': 'Views',
 'Клики': 'Clicks',
 'CTR (%)': 'CTR',
 'Охват': 'Audience',
 'Средняя ставка за 1000 показов (руб.)': 'CPM',
 'Расход (руб., с НДС)': 'Expense',
 'Тип условия': 'Request_Type',
 'sku': 'SKU',
 'Название товара': 'Name',
 'Цена товара (руб.)': 'Price',
 'Заказы': 'Orders',
 'Выручка (руб.)': 'Revenue',
 'Заказы модели': 'Orders_Model',
 'Выручка с заказов модели (руб.)': 'Revenue_Model',
 'Дата': 'DATA',
 'ID заказа': 'Order_ID',
 'Номер заказа': 'Order_Number',
 'Ozon ID': 'Ozon_ID',
 'Ozon ID рекламируемого товара': 'Ozon_ID_Ad_SKU',
 'Артикул': 'Articul',
 'Наименование': 'Name',
 'Количество': 'Orders',
 'Цена продажи': 'Price',
 'Стоимость, руб.': 'Revenue',
 'Ставка, %': 'Search_price_perc',
 'Ставка, руб.': 'Search_price_RUR',
 'Расход, руб.': 'Expense',
 'Unnamed: 1': 'EMPTY',
 'Средняя ставка (руб.)': 'Avrg_bid',
 'Расход за минусом бо

In [25]:
path = r'./data/'

In [13]:
db_data[['id', 'account_id', 'api_id']]

,id,account_id,api_id
0,874555,1,None
1,874556,1,None
2,874557,1,None
3,874558,1,None
4,874559,1,None
...,...,...,...
269440,1301352,2,2490357
269441,1301353,2,2490357
269442,1301354,2,2490357
269443,1301355,2,2490357


In [15]:
db_data['id'].nunique()

269445

In [77]:
# data = working.get_analitics_data()
data = working.db_data

AttributeError: 'db_working' object has no attribute 'db_data'

In [6]:
data['data'].sort_values(ascending=False)[0]

datetime.date(2022, 5, 2)

In [7]:
last_date = working.get_last_date()
last_date

datetime.date(2022, 5, 2)

In [8]:
str(last_date)

'2022-05-02'

In [10]:
api_keys = working.get_keys()

Загрузка api_keys
Доступ к таблице запрещен


In [23]:
data[['id', 'name']]

,id,name
0,874555,Уличная беспроводная WiFi камера видеонаблюден...
1,874556,Беспроводная поворотная IP WiFi камера видеона...
2,874557,Беспроводная IP WiFi камера видеонаблюдения Iv...
3,874558,nan
4,874559,Уличная камера видеонаблюдения Ivideon Bullet ...
...,...,...
269440,1301352,None
269441,1301353,None
269442,1301354,None
269443,1301355,None


In [39]:
for a, keys in data[['id', 'name']].iterrows():
#     print(a)
    print(keys[0])
    print(keys[1])
    break

874555
Уличная беспроводная WiFi камера видеонаблюдения Ivideon Bullet / Пыле-влагозащита IP67 / Встроенный микрофон / Ночной режим


In [ ]:
ozon = Ozon_performance(client_id, client_secret)

In [2]:
# параметры подключения к БД
db_access = """host=rc1b-itt1uqz8cxhs0c3d.mdb.yandexcloud.net\
            port=6432\
            sslmode=verify-full\
            dbname=market_db\
            user=sfedyusnin\
            password=Qazwsx123Qaz\
            target_session_attrs=read-write"""

In [3]:
# проверка доступа к БД
conn = psycopg2.connect(db_access)
q = conn.cursor()
q.execute('SELECT version()')

print(q.fetchone())

conn.close()

('PostgreSQL 14.1 (Ubuntu 14.1-201-yandex.52142.672784f35a) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [4]:
# необходимые запросы к БД
api_keys_resp = 'SELECT * FROM account_list'
keys_dt_cols_resp = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'account_list'"
an_dt_resp = 'SELECT * FROM analitics_data2'
an_dt_cols_resp = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'analitics_data2'"

In [5]:
# an_dt = []
# with closing(psycopg2.connect(db_access)) as conn:
#     with conn.cursor() as cursor:
#         cursor.execute(an_dt_resp)
#         for row in cursor:
# #             print(row)
#             an_dt.append(row)

In [6]:
# an_dt_cols = []
# with closing(psycopg2.connect(db_access)) as conn:
#     with conn.cursor() as cursor:
#         cursor.execute(an_dt_cols_resp)
#         for row in cursor:
# #             print(row[0])
#             an_dt_cols.append(row[0])

In [7]:
an_data = pd.read_sql(an_dt_resp, psycopg2.connect(db_access))

In [8]:
# an_data = pd.DataFrame(an_dt)
# an_data.columns = an_dt_cols

In [10]:
# # api_keys_resp
# keys_dt = []
# try:
#     with closing(psycopg2.connect(db_access)) as conn:
#         with conn.cursor() as cursor:
#             cursor.execute(api_keys_resp)
#             for row in cursor:
# #                 print(row)
#                 keys_dt.append(row)
# except:
#     print('Доступ запрещен')
    
# keys_dt_cols = []
# try:
#     with closing(psycopg2.connect(db_access)) as conn:
#         with conn.cursor() as cursor:
#             cursor.execute(an_dt_cols_resp)
#             for row in cursor:
#     #             print(row[0])
#                 keys_dt_cols.append(row[0])
# except:
#     print('Доступ запрещен')

In [11]:
keys_dt = pd.read_sql(api_keys_resp, psycopg2.connect(db_access))

DatabaseError: Execution failed on sql 'SELECT * FROM account_list': permission denied for table account_list


In [19]:
# распаковываем архивы
zip_files = glob.glob(os.path.join(path, "*.zip"))
for file in zip_files:
    print(f'Распаковка {file}')
    with zipfile.ZipFile(file) as zf:
        zf.extractall(path)

Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_0.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_1.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_2.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_3.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_4.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_5.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_6.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_7.zip
Распаковка ./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_8.zip


In [20]:
# формируем список имен файлов отчетов по кампаниям
camp_files = []
# all_files = glob.glob(os.path.join(path, "*.*"))
# for file in all_files:
#     if os.path.basename(file).startswith('camp'):
#         print(file)
#         with zipfile.ZipFile(file) as zf:
#             camp_files+=zf.namelist()
        
csv_files = glob.glob(os.path.join(path, "*.csv"))
for file in csv_files:
    if os.path.basename(file).count('media') == 0 and os.path.basename(file).count('product') == 0 and os.path.basename(file).count('daily') == 0:
        print(file)
        camp_files.append(file)

./data\1070352_01.07.2022-04.07.2022.csv
./data\1092333_01.07.2022-04.07.2022.csv
./data\1168542_01.07.2022-04.07.2022.csv
./data\1266512_01.07.2022-04.07.2022.csv
./data\2471117-1649840283864@advertising.performance.ozon.ru_campaigns_9.csv
./data\370877_01.07.2022-04.07.2022.csv
./data\370884_01.07.2022-04.07.2022.csv
./data\376752_01.07.2022-04.07.2022.csv
./data\376753_01.07.2022-04.07.2022.csv
./data\380229_01.07.2022-04.07.2022.csv
./data\449864_01.07.2022-04.07.2022.csv
./data\467037_01.07.2022-04.07.2022.csv
./data\781143_01.07.2022-04.07.2022.csv
./data\820263_01.07.2022-04.07.2022.csv
./data\835460_01.07.2022-04.07.2022.csv
./data\835659_01.07.2022-04.07.2022.csv
./data\916272_01.07.2022-04.07.2022.csv
./data\916273_01.07.2022-04.07.2022.csv
./data\941148_01.07.2022-04.07.2022.csv


In [21]:
# функция для обработки датасетов отчетов по кампаниям
def camp_read_trans(path):
    data = pd.read_csv(path, sep=';')
    data = data.reset_index()
    
    camp = data.keys()[-1].split(',')[0].split()[-1]
#     period = data.keys()[-1].split(',')[1].split()[-1]
    
    data.columns = data[0:1].values.tolist()[0]
    data.drop(index=0, inplace=True)
    data.drop(data.tail(1).index, inplace=True)
    
    data['Кампания'] = camp
#     data['Период'] = period
    
    data = data[data.columns[-1:].tolist()+data.columns[:-1].tolist()]
    data = data[data.columns.dropna()]
    data = data.dropna(axis=0, how='any', thresh=10)
    return(data)

In [22]:
camp_data = []
for file in camp_files:
    camp_data.append(camp_read_trans(file))

In [23]:
camp_dataset = pd.concat(camp_data, axis=0).reset_index().drop('index', axis=1)

In [24]:
camp_dataset

,Кампания,День,Тип условия,Условие показа,Платформа,Показы,Клики,CTR (%),Охват,Средняя ставка за 1000 показов (руб.),...,Ozon ID,Ozon ID рекламируемого товара,Артикул,Наименование,Количество,Цена продажи,"Стоимость, руб.","Ставка, %","Ставка, руб.","Расход, руб."
0,1070352,01.07.2022,Категория,Безопасность ребенка,Мобильное приложение,8,0,"0,00",8,"200,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1070352,01.07.2022,Категория,Видеонаблюдение,Мобильное приложение,79,2,"2,53",81,"200,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1070352,01.07.2022,Категория,Видеонаблюдение,Десктоп,47,0,"0,00",34,"200,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1070352,01.07.2022,Категория,Камера видеонаблюдения для дома,Мобильное приложение,115,8,"6,96",108,"200,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1070352,01.07.2022,Категория,Камера видеонаблюдения для дома,Десктоп,28,0,"0,00",23,"200,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,835659,04.07.2022,NaN,Радио- и видеоняни,NaN,102,2,"1,96",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1380,835659,04.07.2022,NaN,Радио- и видеоняни,NaN,44,0,"0,00",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1381,835659,04.07.2022,NaN,Радио- и видеоняни,NaN,131,1,"0,76",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1382,835659,04.07.2022,NaN,NaN,NaN,4,0,"0,00",NaN,"0,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# an_data.sample(5)

In [26]:
camp_dataset.columns

Index(['Кампания', 'День', 'Тип условия', 'Условие показа', 'Платформа',
       'Показы', 'Клики', 'CTR (%)', 'Охват',
       'Средняя ставка за 1000 показов (руб.)', 'Расход (руб., с НДС)',
       'Баннер', 'Тип страницы', 'sku', 'Название товара',
       'Цена товара (руб.)', 'Средняя ставка (руб.)', 'Заказы',
       'Выручка (руб.)', 'Заказы модели', 'Выручка с заказов модели (руб.)',
       'Дата', 'ID заказа', 'Номер заказа', 'Ozon ID',
       'Ozon ID рекламируемого товара', 'Артикул', 'Наименование',
       'Количество', 'Цена продажи', 'Стоимость, руб.', 'Ставка, %',
       'Ставка, руб.', 'Расход, руб.'],
      dtype='object')

In [27]:
an_data.columns

Index(['id', 'banner', 'pagetype', 'viewtype', 'platfrom', 'request_type',
       'sku', 'name', 'order_id', 'order_number', 'ozon_id', 'ozon_id_ad_sku',
       'articul', 'empty', 'account_id', 'views', 'clicks', 'audience',
       'exp_bonus', 'actionnum', 'avrg_bid', 'search_price_rur',
       'search_price_perc', 'price', 'orders', 'revenue_model', 'orders_model',
       'revenue', 'expense', 'cpm', 'ctr', 'data', 'api_id'],
      dtype='object')

In [55]:
an_data['empty'].value_counts()

nan    57919
Name: empty, dtype: int64

In [57]:
an_data['empty'].unique()

array([None, 'nan'], dtype=object)

In [60]:
an_data[['id', 'account_id', 'api_id']]

,id,account_id,api_id
0,874555,1,None
1,874556,1,None
2,874557,1,None
3,874558,1,None
4,874559,1,None
...,...,...,...
269440,1301352,2,2490357
269441,1301353,2,2490357
269442,1301354,2,2490357
269443,1301355,2,2490357


In [63]:
ozon.client_id

'2471117-1649840283864@advertising.performance.ozon.ru'

In [64]:
ozon.client_secret

'aydWhTlynReq4FiDELQhpWC2-j2kfE7n80uOO_ByHt-0A7zAlskxUQFyCa_cp33W0lPvsNZIzwXoSaxh2Q'

In [66]:
an_data['account_id'].value_counts()

2      175479
1       61861
107     16479
146     15626
Name: account_id, dtype: int64

In [67]:
an_data['api_id'].value_counts()

2490357                                                  48457
2506936                                                  32105
2471117                                                  18179
2490357-1651245342742                                    14906
2471117-1649840283864                                     3133
2471117-1649840283864@advertising.performance.ozon.ru      323
Name: api_id, dtype: int64

In [76]:
an_data

NameError: name 'an_data' is not defined

In [28]:
# сопоставляем имена вручную

# {'id': 'Кампания', 
# 'banner': }

In [28]:
from_xl = ['День', 'Баннер', 'Тип страницы', 'Условие показа', 'Платформа', 'Показы', 'Клики', 'CTR (%)', 'Охват',
'Средняя ставка за 1000 показов (руб.)', 'Расход (руб., с НДС)', 'Тип условия', 'sku', 'Название товара',
'Цена товара (руб.)', 'Заказы', 'Выручка (руб.)', 'Заказы модели', 'Выручка с заказов модели (руб.)',
'Дата', 'ID заказа', 'Номер заказа', 'Ozon ID', 'Ozon ID рекламируемого товара', 'Артикул',
'Наименование', 'Количество', 'Цена продажи', 'Стоимость, руб.', 'Ставка, %', 'Ставка, руб.',
'Расход, руб.', 'Unnamed: 1', 'Средняя ставка (руб.)', 'Расход за минусом бонусов (руб., с НДС)']

from_db = ['DATA', 'Banner', 'PageType', 'ViewType', 'Platfrom', 'Views', 'Clicks', 'CTR', 'Audience', 'CPM',
'Expense', 'Request_Type', 'SKU', 'Name', 'Price', 'Orders', 'Revenue', 'Orders_Model', 'Revenue_Model',
'DATA', 'Order_ID', 'Order_Number', 'Ozon_ID', 'Ozon_ID_Ad_SKU', 'Articul', 'Name', 'Orders', 'Price',
'Revenue', 'Search_price_perc', 'Search_price_RUR', 'Expense', 'EMPTY', 'Avrg_bid', 'exp_bonus']

mapping_data = dict(zip(from_xl, from_db))

In [47]:
len(from_xl)

35

In [48]:
len(from_db)

35

In [49]:
len(an_data.columns)

33

In [50]:
len(camp_dataset.columns)

34

In [34]:
mapping_data = dict(zip(from_xl, from_db))

In [35]:
mapping_data

{'День': 'DATA',
 'Баннер': 'Banner',
 'Тип страницы': 'PageType',
 'Условие показа': 'ViewType',
 'Платформа': 'Platfrom',
 'Показы': 'Views',
 'Клики': 'Clicks',
 'CTR (%)': 'CTR',
 'Охват': 'Audience',
 'Средняя ставка за 1000 показов (руб.)': 'CPM',
 'Расход (руб., с НДС)': 'Expense',
 'Тип условия': 'Request_Type',
 'sku': 'SKU',
 'Название товара': 'Name',
 'Цена товара (руб.)': 'Price',
 'Заказы': 'Orders',
 'Выручка (руб.)': 'Revenue',
 'Заказы модели': 'Orders_Model',
 'Выручка с заказов модели (руб.)': 'Revenue_Model',
 'Дата': 'DATA',
 'ID заказа': 'Order_ID',
 'Номер заказа': 'Order_Number',
 'Ozon ID': 'Ozon_ID',
 'Ozon ID рекламируемого товара': 'Ozon_ID_Ad_SKU',
 'Артикул': 'Articul',
 'Наименование': 'Name',
 'Количество': 'Orders',
 'Цена продажи': 'Price',
 'Стоимость, руб.': 'Revenue',
 'Ставка, %': 'Search_price_perc',
 'Ставка, руб.': 'Search_price_RUR',
 'Расход, руб.': 'Expense',
 'Unnamed: 1': 'EMPTY',
 'Средняя ставка (руб.)': 'Avrg_bid',
 'Расход за минусом бо

In [29]:
# формируем список имен файлов отчетов по media, product
mp_files = []
csv_files = glob.glob(os.path.join(path, "*.csv"))
for file in csv_files:
    if os.path.basename(file).count('media') > 0 or os.path.basename(file).count('product') > 0:
        mp_files.append(file)

In [30]:
mp_files

['./data\\2471117-1649840283864@advertising.performance.ozon.ru_media_2022-07-01-2022-07-04.csv',
 './data\\2471117-1649840283864@advertising.performance.ozon.ru_product_2022-07-01-2022-07-04.csv']

In [31]:
def mp_read_trans(path):
    data = pd.read_csv(path, sep=';')    
    period = path.split('_')[1].split('.')[0]   
    data['Период'] = period
    return(data)  

In [32]:
mp_data = []
for file in mp_files:
    mp_data.append(mp_read_trans(file))

In [33]:
mp_data[0].shape

(9, 18)

In [34]:
mp_data[1].shape

(6, 15)

In [35]:
mp_dataset = pd.concat(mp_data, axis=0).reset_index().drop('index', axis=1)

In [36]:
mp_dataset

,ID,Название,Формат,Статус,"Дневной бюджет, ₽","Бюджет, ₽",Приоритет,"Расход, ₽",Показы,Клики,"Средняя ставка, ₽","Ср. цена 1000 показов, ₽","CTR, %","Ср. цена клика, ₽","Заказы, шт.","Заказы, руб",ДРР,Период
0,376752,Брендовая полка,BRAND_SHELF,inactive,1000,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
1,376753,Баннер,BANNER,inactive,1000,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
2,380229,Баннер,BANNER,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
3,820263,Брендовая полка 1.2,BRAND_SHELF,running,1000,0.0,auction,1101.600,5508,228,0.200000,200.000000,"0,04",4.831579,0,0,"0,00",media
4,835460,Брендовая полка 2,BRAND_SHELF,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
5,941148,Брендовая полка,BRAND_SHELF,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
6,1070352,Брендовая полка 1,BRAND_SHELF,running,1000,0.0,auction,1281.600,6408,292,0.200000,200.000000,"0,05",4.389041,0,0,"0,00",media
7,1092333,Сегмент для подвижения,BANNER,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
8,1266512,Баннер на главной,BANNER,inactive,1000,5000.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",media
9,370877,Карточка товара видеокамеры,NaN,running,1000,NaN,NaN,577.400,4052,54,0.152017,142.497532,"0,01",10.692593,0,0,"0,00",product


In [37]:
mp_dataset.columns

Index(['ID', 'Название', 'Формат', 'Статус', 'Дневной бюджет, ₽', 'Бюджет, ₽',
       'Приоритет', 'Расход, ₽', 'Показы', 'Клики', 'Средняя ставка, ₽',
       'Ср. цена 1000 показов, ₽', 'CTR, %', 'Ср. цена клика, ₽',
       'Заказы, шт.', 'Заказы, руб', 'ДРР', 'Период'],
      dtype='object')

In [38]:
# формируем список имен файлов отчетов daily
d_files = []
csv_files = glob.glob(os.path.join(path, "*.csv"))
for file in csv_files:
    if os.path.basename(file).count('daily') > 0:
        d_files.append(file)

In [39]:
d_files

['./data\\2471117-1649840283864@advertising.performance.ozon.ru_daily_2022-07-01-2022-07-04.csv']

In [40]:
d_data = []
for file in d_files:
    d_data.append(pd.read_csv(file, sep=';'))

In [41]:
d_data[0].shape

(34, 9)

In [42]:
d_dataset = pd.concat(d_data, axis=0).reset_index().drop('index', axis=1)
d_dataset = d_dataset.drop_duplicates()

In [43]:
d_dataset

,ID,Название,Дата,Показы,Клики,"Расход, ₽","Средняя ставка, ₽","Заказы, шт.","Заказы, руб"
0,370877,Карточка товара видеокамеры,2022-07-01,1313,19,"205,20","0,18",0,"0,00"
1,370877,Карточка товара видеокамеры,2022-07-02,939,16,"124,40","0,14",0,"0,00"
2,370877,Карточка товара видеокамеры,2022-07-03,842,10,"113,10","0,14",0,"0,00"
3,370877,Карточка товара видеокамеры,2022-07-04,958,9,"134,70","0,15",0,"0,00"
4,467037,Поиск cute 2,2022-07-01,1126,30,"547,42","0,49",0,"0,00"
5,467037,Поиск cute 2,2022-07-02,1108,34,"550,49","0,50",1,"3990,00"
6,467037,Поиск cute 2,2022-07-03,1246,30,"613,17","0,49",0,"0,00"
7,467037,Поиск cute 2,2022-07-04,1279,31,"602,69","0,47",1,"3990,00"
8,835659,Карточка товара видеоняня,2022-07-01,355,3,"105,60","0,30",0,"0,00"
9,835659,Карточка товара видеоняня,2022-07-02,323,1,"95,40","0,30",0,"0,00"
